In [1]:


import functools
from typing import Callable, Dict, List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scr.activation_funcs import ReLU, identity, sign, tanh
from scr.binary_operators import (
    get_x,
    get_y,
    operators_add,
    operators_diff,
    operators_max,
    operators_min,
    operators_multiple,
    x_is_greater_than_y,
)
from scr.utils import create_formulae,_calc_ols_func,rolling_window
from tqdm.notebook import tqdm

In [2]:
def dynamics(y_t: np.ndarray, sigma: float):

    a, b = y_t
    y_next = np.zeros_like(y_t)
    size = y_t.shape[1]

    y_next[0] = 1.0 * np.tanh(a) + 0.8 * a * b + 1.0 * b - 1.0 * ReLU(
        np.minimum(a, b)) + sigma * np.random.randn(size)
    y_next[1] = 0.6 * np.sign(b) + 0.5 * a * b - 1.0 * np.maximum(
        a, b) + sigma * np.random.randn(size)

    return y_next


T_total = 2000
sigma = 0.1
y = np.zeros((2, T_total))
# axis=0 为时间 axis=1为股票
data = dynamics(y, sigma).T

In [3]:
class Trader(object):
    def __init__(self,
                 M: int,
                 A: List[Callable],
                 O: List[Callable],
                 stock_num: int,
                 max_lag: int = 9,
                 l: int = 1,
                 seed: int = None) -> None:

        activation_funcs: List[Callable] = [identity, ReLU, sign, tanh]
        binary_operators: List[Callable] = [
            operators_max, operators_min, operators_add, operators_diff, get_x,
            get_y, operators_multiple, x_is_greater_than_y
        ]

        if seed:
            np.random.seed(seed)

        # 生成公式
        ## List中的下表对应的是数据中的股票
        self.formulate: List[List[Callable]] = [
            create_formulae(M,
                            activation_funcs,
                            binary_operators,
                            stock_num=stock_num,
                            max_lag=max_lag,
                            l=l) for _ in range(stock_num)
        ]
        # 生成权重
        ## List中的下表对应的是数据中的股票
        self.weight: List[np.ndarray] = self.get_randn_weight()

    def get_randn_weight(self) -> List[np.ndarray]:
        """随机生成权重"""

        return [np.random.randn(len(formula)) for formula in self.formulate]

    def calc_factors(self, data: np.ndarray) -> None:

        factors: List = []  # 储存因子

        for stocki_formula in self.formulate:

            p: List = [formula(data) for formula in stocki_formula]

            p: np.ndarray = np.vstack(p).T

            factors.append(p)

        # 获取因子
        self.factors: List[np.ndarray] = factors

    def fit(self, data: np.ndarray) -> np.ndarray:

        self.calc_factors(data)
        pred: List = [
            factor @ w for factor, w in zip(self.factors, self.weight)
        ]

        return np.vstack(pred).T

    def learn(self, endog: np.ndarray, func: Callable = _calc_ols_func) -> None:

        for num, factor in enumerate(self.factors):

            self.weight[num] = _calc_ols_func(factor, endog)

    def predict(self) -> np.ndarray:

        y_pred: List = [
            factor[-1] @ w for factor, w in zip(self.factors, self.weight)
        ]
        return np.vstack(y_pred).T
    
    def calc_cumulative_err(self,real_value:np.ndarray):
        
        y_pred:List = [
            factor @ w for factor, w in zip(self.factors, self.weight)
        ]
        
        y_pred:np.ndarray = np.vstack(y_pred).T
        
        errors:np.ndarry = np.sqrt(np.square(y_pred - real_value).mean())
        
        self.cumulative_err:np.ndarray = errors


class Company(object):
    def __init__(self,
                 stock_names: List[str],
                 M: int,
                 max_lag: int,
                 l: int,
                 activation_funcs: List[Callable],
                 binary_operators: List[Callable],
                 traders_num: int,
                 Q: float = 0.5,
                 seed: int = None) -> None:

        if seed:
            np.random.seed(seed)

        self.traders: List[Trader] = [
            Trader(M, activation_funcs, binary_operators, len(stock_names),
                   max_lag, l) for _ in range(traders_num)
        ]

    def fit(self, train_data: np.ndarray):

        datas:List[np.ndarray] = rolling_window(train_data,self.max_lag+self.l)
        
        for data in datas:
            
            for trader in self.traders:
                
                for stock_i_formula in trader:

In [4]:
activation_funcs:List[Callable] = [identity, ReLU, sign, tanh]
binary_operators:List[Callable] = [
    operators_max, operators_min, operators_add,
                                  operators_diff, get_x, get_y, operators_multiple, x_is_greater_than_y
]

# model = Company([1,2],10,9,1,activation_funcs,binary_operators,10,0.5,42)

In [5]:

trader = [Trader(2,activation_funcs,binary_operators,2) for _ in range(10)]